# How to stream data from within a tool
- https://langchain-ai.github.io/langgraph/how-tos/streaming-events-from-within-tools/
- If graph use tools that use LLMs or any other streaming API, you might want to surface partial results. Especially if the tool takes longer time to run.
- Technique 1: `stream_mode="custom"` and `get_stream_writer()` method.
- Technique 2: `stream_mode="messages"`

### Streaming custom data

In [7]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

from langgraph.prebuilt import create_react_agent
from langgraph.config import get_stream_writer

@tool
async def get_items(place: str) -> str:
    """Use this tool to list items one might find in a place you're asked about."""
    writer = get_stream_writer()

    # this can be replaced with any actual streaming logic that you might have
    items = ["books", "penciles", "pictures"]
    for chunk in items:
        writer({"custom_tool_data": chunk})

    return ", ".join(items)

llm = ChatOpenAI(model_name="gpt-4o-mini")
tools = [get_items]
agent = create_react_agent(llm, tools=tools)

In [8]:
inputs = {
    "messages": [
        {"role": "user", "content": "What items are in the office?"}
    ]
}

async for chunk in agent.astream(inputs, stream_mode="custom"):
    print(chunk)

{'custom_tool_data': 'books'}
{'custom_tool_data': 'penciles'}
{'custom_tool_data': 'pictures'}


### Streaming LLM tokens

In [10]:
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import AIMessageChunk

@tool
async def get_items(
    place: str,
    config: RunnableConfig
) -> str:
    """
    Use this tools to list items one might find in a place you're asked about.
    """
    response = await llm.ainvoke(
        [
            {
                "role": "user",
                "content": f"Can you tell me what kind of items i might find in the following place: '{place}'. "
                    "List at least 3 such items separating them by a comma. And include a brief description of each item."
            }
        ],
        config
    )

    return response.content


tools = [get_items]
agent = create_react_agent(llm, tools=tools)

In [11]:
inputs = {
    "messages": [
        {"role": "user", "content": "what items are in the bedroom?"}
    ]
}

async for msg, metadata in agent.astream(
    inputs,
    stream_mode="messages",
):
    if (
        isinstance(msg, AIMessageChunk)
        and msg.content
        # Stream all messages from the tool node
        and metadata["langgraph_node"] == "tools"
    ):
        print(msg.content, end="|", flush=True)

Sure|!| Here| are| three| items| you| might| find| in| a| bedroom|:

|1|.| **|Bed|**|:| The| centerpiece| of| a| bedroom|,| typically| consisting| of| a| mattress| and| a| bed| frame|.| It| is| used| for| sleeping|,| resting|,| and| relaxation|.| Beds| come| in| various| sizes|,| such| as| twin|,| full|,| queen|,| and| king|,| and| can| be| adorned| with| pillows| and| bedding| for| comfort|.

|2|.| **|D|resser|**|:| A| piece| of| furniture| with| multiple| drawers| used| for| storing| clothing| and| personal| items|.| Dress|ers| often| have| a| flat| surface| on| top| where| items| like| jewelry| boxes|,| photographs|,| or| decorative| items| can| be| placed|.| They| help| keep| the| bedroom| organized| and| tidy|.

|3|.| **|Night|stand|**|:| A| small| table| typically| located| beside| the| bed|,| used| to| hold| essentials| like| a| lamp|,| alarm| clock|,| books|,| or| a| glass| of| water|.| Night|stands| provide| convenient| storage| and| surface| space| for| items| you| may| need|

### Example without LangChain

In [12]:
import operator
import json

from typing import TypedDict
from typing_extensions import Annotated
from langgraph.graph import StateGraph, START

from openai import AsyncOpenAI

openai_client = AsyncOpenAI()
model_name = "gpt-4o-mini"


async def stream_tokens(model_name: str, messages: list[dict]):
    response = await openai_client.chat.completions.create(
        messages=messages, model=model_name, stream=True
    )
    role = None
    async for chunk in response:
        delta = chunk.choices[0].delta

        if delta.role is not None:
            role = delta.role

        if delta.content:
            yield {"role": role, "content": delta.content}


# this is our tool
async def get_items(place: str) -> str:
    """Use this tool to list items one might find in a place you're asked about."""
    writer = get_stream_writer()
    response = ""
    async for msg_chunk in stream_tokens(
        model_name,
        [
            {
                "role": "user",
                "content": (
                    "Can you tell me what kind of items "
                    f"i might find in the following place: '{place}'. "
                    "List at least 3 such items separating them by a comma. "
                    "And include a brief description of each item."
                ),
            }
        ],
    ):
        response += msg_chunk["content"]
        writer(msg_chunk)

    return response


class State(TypedDict):
    messages: Annotated[list[dict], operator.add]


# this is the tool-calling graph node
async def call_tool(state: State):
    ai_message = state["messages"][-1]
    tool_call = ai_message["tool_calls"][-1]

    function_name = tool_call["function"]["name"]
    if function_name != "get_items":
        raise ValueError(f"Tool {function_name} not supported")

    function_arguments = tool_call["function"]["arguments"]
    arguments = json.loads(function_arguments)

    function_response = await get_items(**arguments)
    tool_message = {
        "tool_call_id": tool_call["id"],
        "role": "tool",
        "name": function_name,
        "content": function_response,
    }
    return {"messages": [tool_message]}


graph = (
    StateGraph(State)
    .add_node(call_tool)
    .add_edge(START, "call_tool")
    .compile()
)

In [13]:
inputs = {
    "messages": [
        {
            "content": None,
            "role": "assistant",
            "tool_calls": [
                {
                    "id": "1",
                    "function": {
                        "arguments": '{"place":"bedroom"}',
                        "name": "get_items",
                    },
                    "type": "function",
                }
            ],
        }
    ]
}

async for chunk in graph.astream(
    inputs,
    stream_mode="custom",
):
    print(chunk["content"], end="|", flush=True)

Sure|!| Here| are| three| items| commonly| found| in| a| bedroom|:

|1|.| **|Bed|**|:| A| piece| of| furniture| typically| consisting| of| a| mattress| and| a| frame|,| used| for| sleeping|.| Beds| can| vary| in| size| (|such| as| twin|,| full|,| queen|,| or| king|)| and| can| come| with| various| styles| and| additional| features| like| storage| drawers| or| elevated| frames|.

|2|.| **|D|resser|**|:| A| storage| unit| with| multiple| drawers| used| for| organizing| clothing| and| personal| items|.| Dress|ers| often| serve| as| a| surface| for| additional| decorative| items| and| may| also| come| with| a| mirror| attached| or| positioned| above| it|.

|3|.| **|Night|stand|**|:| A| small| table| or| cabinet| positioned| beside| the| bed|,| used| to| hold| items| like| lamps|,| alarm| clocks|,| books|,| or| personal| items| for| easy| access| during| the| night|.| Night|stands| can| vary| in| design| and| materials|,| complement|ing| the| bedroom|'s| overall| decor|.|